In [8]:
import os, sys, shutil
import torch
from tqdm import tqdm
import numpy as np
import pandas as pd
import matplotlib as plt
from PIL import Image
import nibabel as nib
os.chdir("/home/naxos2-raid25/ojeda040/local/ojeda040/MindEye_Imagery/src")

from utils import *
# # from autoencoder import AutoEncoder
# from gnet8_encoder import GNet8_Encoder
# from matplotlib.lines import Line2D
# import matplotlib as mpl
# import math
# import matplotlib.image as mpimg
# import random
os.chdir("..")

In [12]:
def get_kastner_rois(subject, data_path):
    kastner_labels = f"{data_path}/nsddata/freesurfer/subj0{subject}/label/Kastner2015.mgz.ctab"
    brainmask_inflated = nib.load(f"{data_path}/nsddata/ppdata/subj0{subject}/func1pt8mm/roi/brainmask_inflated_1.0.nii").get_fdata()
    brainmask_inflated = np.nan_to_num(brainmask_inflated)
    brainmask_inflated = np.where(brainmask_inflated==1.0, True, False)
    kastner_mask = nib.load(f"{data_path}/nsddata/ppdata/subj0{subject}/func1pt8mm/roi/Kastner2015.nii.gz").get_fdata()
    kastner_mask = kastner_mask[brainmask_inflated]
    with open(kastner_labels, 'r') as file:
        labels = file.read().splitlines()
    kastner_mask_labeled = {}
    for label in labels[1:]:
        label = label.split(" ")
        kastner_mask_labeled[label[1].strip()] = np.where(kastner_mask==int(label[0]), True, False)
    kastner_mask_labeled["V1"] = np.logical_or(kastner_mask_labeled["V1v"], kastner_mask_labeled["V1d"])
    kastner_mask_labeled["V2"] = np.logical_or(kastner_mask_labeled["V2v"], kastner_mask_labeled["V2d"])
    kastner_mask_labeled["V3"] = np.logical_or(kastner_mask_labeled["V3v"], kastner_mask_labeled["V3d"])
    del kastner_mask_labeled["V1v"]
    del kastner_mask_labeled["V1d"]
    del kastner_mask_labeled["V2v"]
    del kastner_mask_labeled["V2d"]
    del kastner_mask_labeled["V3v"]
    del kastner_mask_labeled["V3d"]
    nsd_general = nib.load(f"{data_path}/nsddata/ppdata/subj0{subject}/func1pt8mm/roi/nsdgeneral.nii.gz").get_fdata()
    nsd_general = np.nan_to_num(nsd_general)
    nsd_general = nsd_general[brainmask_inflated]
    nsd_general = np.where(nsd_general==1.0, True, False)
    kastner_mask_labeled['nsd_general'] = nsd_general
    return kastner_mask_labeled

def get_hcp_mmp1_rois(subject, data_path):
    mmp1_labels = f"{data_path}/nsddata/freesurfer/subj0{subject}/label/HCP_MMP1.mgz.ctab"
    brainmask_inflated = nib.load(f"{data_path}/nsddata/ppdata/subj0{subject}/func1pt8mm/roi/brainmask_inflated_1.0.nii").get_fdata()
    brainmask_inflated = np.nan_to_num(brainmask_inflated)
    brainmask_inflated = np.where(brainmask_inflated==1.0, True, False)
    mmp1_mask = nib.load(f"{data_path}/nsddata/ppdata/subj0{subject}/func1pt8mm/roi/HCP_MMP1.nii.gz").get_fdata()
    mmp1_mask = mmp1_mask[brainmask_inflated]
    with open(mmp1_labels, 'r') as file:
        labels = file.read().splitlines()
    mmp1_mask_labeled = {}
    for label in labels[1:]:
        label = label.split(" ")
        mmp1_mask_labeled[label[1].strip()] = np.where(mmp1_mask==int(label[0]), True, False)
    nsd_general = nib.load(f"{data_path}/nsddata/ppdata/subj0{subject}/func1pt8mm/roi/nsdgeneral.nii.gz").get_fdata()
    nsd_general = np.nan_to_num(nsd_general)
    nsd_general = nsd_general[brainmask_inflated]
    nsd_general = np.where(nsd_general==1.0, True, False)
    mmp1_mask_labeled['nsd_general'] = nsd_general
    return mmp1_mask_labeled

def get_thalamus_rois(subject, data_path):
    thalamus_labels = f"{data_path}/nsddata/templates/thalamus.ctab"
    brainmask_inflated = nib.load(f"{data_path}/nsddata/ppdata/subj0{subject}/func1pt8mm/roi/brainmask_inflated_1.0.nii").get_fdata()
    brainmask_inflated = np.nan_to_num(brainmask_inflated)
    brainmask_inflated = np.where(brainmask_inflated==1.0, True, False)
    thalamus_mask = nib.load(f"data/nsddata/ppdata/subj0{subject}/func1pt8mm/roi/thalamus.nii.gz").get_fdata()
    thalamus_mask = thalamus_mask[brainmask_inflated]
    with open(thalamus_labels, 'r') as file:
        labels = file.read().splitlines()
    thalamus_mask_labeled = {}
    for label in labels[1:]:
        label = label.split(" ")
        thalamus_mask_labeled[label[1].strip()] = np.where(thalamus_mask==int(label[0]), True, False)
    return thalamus_mask_labeled

def get_mtl_rois(subject, data_path):
    mtl_labels = f"{data_path}/nsddata/templates/MTL.ctab"
    brainmask_inflated = nib.load(f"{data_path}/nsddata/ppdata/subj0{subject}/func1pt8mm/roi/brainmask_inflated_1.0.nii").get_fdata()
    brainmask_inflated = np.nan_to_num(brainmask_inflated)
    brainmask_inflated = np.where(brainmask_inflated==1.0, True, False)
    mtl_mask = nib.load(f"data/nsddata/ppdata/subj0{subject}/func1pt8mm/roi/MTL.nii.gz").get_fdata()
    mtl_mask = mtl_mask[brainmask_inflated]
    with open(mtl_labels, 'r') as file:
        labels = file.read().splitlines()
    mtl_mask_labeled = {}
    for label in labels[1:]:
        label = label.split(" ")
        mtl_mask_labeled[label[1].strip()] = np.where(mtl_mask==int(label[0]), True, False)
    return mtl_mask_labeled

def get_streams_rois(subject, data_path):
    streams_labels = f"{data_path}/nsddata/freesurfer/subj0{subject}/label/streams.mgz.ctab"
    brainmask_inflated = nib.load(f"{data_path}/nsddata/ppdata/subj0{subject}/func1pt8mm/roi/brainmask_inflated_1.0.nii").get_fdata()
    brainmask_inflated = np.nan_to_num(brainmask_inflated)
    brainmask_inflated = np.where(brainmask_inflated==1.0, True, False)
    streams_mask = nib.load(f"{data_path}/nsddata/ppdata/subj0{subject}/func1pt8mm/roi/streams.nii.gz").get_fdata()
    streams_mask = streams_mask[brainmask_inflated]
    with open(streams_labels, 'r') as file:
        labels = file.read().splitlines()
    streams_mask_labeled = {}
    for label in labels[1:]:
        label = label.split(" ")
        streams_mask_labeled[label[1].strip()] = np.where(streams_mask==int(label[0]), True, False)
    nsd_general = nib.load(f"{data_path}/nsddata/ppdata/subj0{subject}/func1pt8mm/roi/nsdgeneral.nii.gz").get_fdata()
    nsd_general = np.nan_to_num(nsd_general)
    nsd_general = nsd_general[brainmask_inflated]
    nsd_general = np.where(nsd_general==1.0, True, False)
    streams_mask_labeled['nsd_general'] = nsd_general
    return streams_mask_labeled

def get_floc_rois(subject, data_path, floc_functionality='faces'):
    floc_labels = f"{data_path}/nsddata/freesurfer/subj0{subject}/label/floc-{floc_functionality}.mgz.ctab"
    brainmask_inflated = nib.load(f"{data_path}/nsddata/ppdata/subj0{subject}/func1pt8mm/roi/brainmask_inflated_1.0.nii").get_fdata()
    brainmask_inflated = np.nan_to_num(brainmask_inflated)
    brainmask_inflated = np.where(brainmask_inflated==1.0, True, False)
    floc_mask = nib.load(f"{data_path}/nsddata/ppdata/subj0{subject}/func1pt8mm/roi/floc-{floc_functionality}.nii.gz").get_fdata()
    floc_mask = floc_mask[brainmask_inflated]
    with open(floc_labels, 'r') as file:
        labels = file.read().splitlines()
    floc_mask_labeled = {}
    for label in labels[1:]:
        label = label.split(" ")
        floc_mask_labeled[label[1].strip()] = np.where(floc_mask==int(label[0]), True, False)
    return floc_mask_labeled

# kastner_mask_labeled = get_kastner_rois(1, "data/")
# print(kastner_mask_labeled.keys(), np.unique(kastner_mask_labeled['V1'], return_counts=True))

# mm1_mask_labeled = get_hcp_mmp1_rois(1, "data/")
# print(mm1_mask_labeled.keys(), np.unique(mm1_mask_labeled['V1'], return_counts=True))

# thalamus_mask_labeled = get_thalamus_rois(1, "data/")
# print(thalamus_mask_labeled.keys(), np.unique(thalamus_mask_labeled['LGN'], return_counts=True))

# mtl_mask_labeled = get_mtl_rois(1, "data/")
# print(mtl_mask_labeled.keys(), np.unique(mtl_mask_labeled['ERC'], return_counts=True))

streams_mask_labeled = get_streams_rois(1, "dataset/")
print(streams_mask_labeled.keys(), np.unique(streams_mask_labeled['ventral'], return_counts=True))


dict_keys(['early', 'midventral', 'midlateral', 'midparietal', 'ventral', 'lateral', 'parietal', 'nsd_general']) (array([False,  True]), array([230904,   7604]))


In [10]:
with h5py.File("/home/naxos2-raid25/kneel027/home/kneel027/data/mindeye_imagery/kastner_rois.hdf5", 'w') as hdf:
    for subject in range(1, 9):
        subject_str = f"subj{subject:02d}"
        mask_dictionary = get_kastner_masks(subject, "data/")

        # Create a group for each subject
        subject_group = hdf.create_group(subject_str)

        # Iterate through the mask dictionary and create datasets for each mask
        for region, mask in mask_dictionary.items():
            subject_group.create_dataset(region, data=mask)
            


In [ ]:
with h5py.File("/home/naxos2-raid25/kneel027/home/kneel027/data/mindeye_imagery/hcp_mmp1_rois.hdf5", 'w') as hdf:
    for subject in range(1, 9):
        subject_str = f"subj{subject:02d}"
        mask_dictionary = get_hcp_mmp1_rois(subject, "data/")

        # Create a group for each subject
        subject_group = hdf.create_group(subject_str)

        # Iterate through the mask dictionary and create datasets for each mask
        for region, mask in mask_dictionary.items():
            subject_group.create_dataset(region, data=mask)

In [16]:
with h5py.File("/home/naxos2-raid25/kneel027/home/kneel027/data/mindeye_imagery/thalamus_rois.hdf5", 'w') as hdf:
    for subject in range(1, 9):
        subject_str = f"subj{subject:02d}"
        mask_dictionary = get_thalamus_rois(subject, "data/")

        # Create a group for each subject
        subject_group = hdf.create_group(subject_str)

        # Iterate through the mask dictionary and create datasets for each mask
        for region, mask in mask_dictionary.items():
            subject_group.create_dataset(region, data=mask)

In [ ]:
with h5py.File("/home/naxos2-raid25/kneel027/home/kneel027/data/mindeye_imagery/mtl_rois.hdf5", 'w') as hdf:
    for subject in range(1, 9):
        subject_str = f"subj{subject:02d}"
        mask_dictionary = get_mtl_rois(subject, "data/")

        # Create a group for each subject
        subject_group = hdf.create_group(subject_str)

        # Iterate through the mask dictionary and create datasets for each mask
        for region, mask in mask_dictionary.items():
            subject_group.create_dataset(region, data=mask)

In [ ]:
with h5py.File("/home/naxos2-raid25/kneel027/home/kneel027/data/mindeye_imagery/streams_rois.hdf5", 'w') as hdf:
    for subject in range(1, 9):
        subject_str = f"subj{subject:02d}"
        mask_dictionary = get_streams_rois(subject, "data/")

        # Create a group for each subject
        subject_group = hdf.create_group(subject_str)

        # Iterate through the mask dictionary and create datasets for each mask
        for region, mask in mask_dictionary.items():
            subject_group.create_dataset(region, data=mask)

In [16]:
with h5py.File("/home/naxos2-raid25/ojeda040/local/ojeda040/MindEye_Imagery/dataset/floc_words_rois.hdf5", 'w') as hdf:
    for subject in range(1, 9):
        subject_str = f"subj{subject:02d}"
        mask_dictionary = get_floc_rois(subject, "dataset/", floc_functionality='words')

        # Create a group for each subject
        subject_group = hdf.create_group(subject_str)

        # Iterate through the mask dictionary and create datasets for each mask
        for region, mask in mask_dictionary.items():
            subject_group.create_dataset(region, data=mask)